### Transformers  
Code comments copied from `https://github.com/python-engineer/pytorchTutorial/blob/master/10_transformers.py`   

---------
Transforms can be applied to PIL images, tensors, ndarrays, or custom data
during creation of the DataSet
complete list of built-in transforms: 
https://pytorch.org/docs/stable/torchvision/transforms.html
On Images
---------
CenterCrop, Grayscale, Pad, RandomAffine
RandomCrop, RandomHorizontalFlip, RandomRotation
Resize, Scale
On Tensors
----------
LinearTransformation, Normalize, RandomErasing
Conversion
----------
ToPILImage: from tensor or ndrarray
ToTensor : from numpy.ndarray or PILImage
Generic
-------
Use Lambda 
Custom
------
Write own class
Compose multiple Transforms
---------------------------
composed = transforms.Compose([Rescale(256),
                               RandomCrop(224)])

In [2]:
import torch
import torchvision
from torch.utils.data import Dataset
import numpy as np


In [9]:
class WineDataset(Dataset):
    
    def __init__(self, transform=None):
        xy = np.loadtxt('./data/wine/wine.txt', delimiter=',', dtype=np.float32, skiprows=1)
        self.n_samples = xy.shape[0]
        self.transform = transform  # note transform needs to be callable, see the getitem method
        
        # note we are not converting to tensor here, leaving them as np arrays
        self.x = xy[:, 1:]
        self.y = xy[:, [0]] # note the weird syntax
        
    def __getitem__(self, index):
        sample = self.x[index], self.y[index]
        
        if self.transform:
            sample = self.transform(sample)
        return sample
    
    def __len__(self):
        return self.n_samples


#### Some example custome transform classes for fun!

In [10]:
class ToTensor():
    def __call__(self, sample):
        # the only contract we need to fulfill to make a callable class
        inputs, targets = sample
        return torch.from_numpy(inputs), torch.from_numpy(targets)

In [11]:
# Note the ToTensor syntax, it looks like it's being called, but it has to be passing in an callable function
dataset = WineDataset(transform = ToTensor())

In [13]:
first_data = dataset[0]
features, labels = first_data
features, labels

(tensor([1.4230e+01, 1.7100e+00, 2.4300e+00, 1.5600e+01, 1.2700e+02, 2.8000e+00,
         3.0600e+00, 2.8000e-01, 2.2900e+00, 5.6400e+00, 1.0400e+00, 3.9200e+00,
         1.0650e+03]),
 tensor([1.]))

In [14]:
type(features), type(labels)

(torch.Tensor, torch.Tensor)

#### Now if we don't pass in the ToTensor transform to the dataset

In [15]:
dataset = WineDataset()
first_data = dataset[0]
features, labels = first_data
type(features), type(labels)

(numpy.ndarray, numpy.ndarray)

#### Another custom transformer for fun

In [16]:
# This transformer multiply features by a given factor
class MulTransform:
    def __init__(self, factor):
        self.factor = factor
        
    def __call__(self, sample):
        inputs, target = sample
        inputs *= self.factor
        return inputs, target 

#### We can compose transformers, or maybe more like composing functions

In [17]:
# note the syntax again, looks like we are calling something, but with the parens we get a function?
composed = torchvision.transforms.Compose([ToTensor(), MulTransform(2)])
dataset = WineDataset(transform=composed)
first_data = dataset[0]
features, labels = first_data
type(features), type(labels)

(torch.Tensor, torch.Tensor)

In [18]:
features, labels

(tensor([2.8460e+01, 3.4200e+00, 4.8600e+00, 3.1200e+01, 2.5400e+02, 5.6000e+00,
         6.1200e+00, 5.6000e-01, 4.5800e+00, 1.1280e+01, 2.0800e+00, 7.8400e+00,
         2.1300e+03]),
 tensor([1.]))